In [12]:
def dataframe_to_redshift(df, table, schema, bucket="redshift-python-datalake", database='landing_zone', region_name='us-west-2', endpoint='data_lake', object_name=None, db_user='admintreinta', cluster_identifier='redshift-data'):
    # Asegúrate de que todos los argumentos se pasen por nombre
    s3_object_path = dataframe_to_s3(df=df, bucket=bucket, endpoint=endpoint, region_name=region_name, object_name=object_name)
    output = load_s3_to_redshift(table=table, schema=schema, s3_object_path=s3_object_path, database=database, cluster_identifier=cluster_identifier, db_user=db_user, region_name=region_name)
    return output


In [2]:
dataframe_to_redshift()

In [10]:
DATA_1 = query_to_dataframe("SELECT * FROM landing_zone.public.frequency;", cluster_identifier = 'redshift-data', database = "landing_zone", db_user = 'admintreinta')
DATA_1

Current status: FINISHED


,id,months_frequency,discount
0,1,1,0
1,3,12,0.4
2,2,6,0.25


In [13]:
dataframe_to_redshift(DATA_1,'landing_zone','public')


        COPY landing_zone.public.landing_zone
        FROM 's3://redshift-python-datalake/data_lake/2024/03/18/None_2024_03_18_16_07_01_272426.csv.gz'
        IAM_ROLE default
        delimiter ','
        IGNOREHEADER 1
        GZIP
        CSV;
    
Estado actual: FAILED
Error al truncar la tabla: ERROR: Cannot COPY into nonexistent table landing_zone


{'ClusterIdentifier': 'redshift-data',
 'CreatedAt': datetime.datetime(2024, 3, 18, 16, 7, 3, 91000, tzinfo=tzlocal()),
 'Database': 'landing_zone',
 'DbUser': 'admintreinta',
 'Id': 'd88c20cb-9376-47c7-acb2-d47f023797f1',
 'ResponseMetadata': {'RequestId': 'd88c20cb-9376-47c7-acb2-d47f023797f1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd88c20cb-9376-47c7-acb2-d47f023797f1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '160',
   'date': 'Mon, 18 Mar 2024 21:07:02 GMT'},
  'RetryAttempts': 0}}